Load data

In [1]:
!pip freeze

appnope==0.1.0
attrs==19.3.0
backcall==0.1.0
bleach==3.1.4
certifi==2020.4.5.1
cycler==0.10.0
Cython==0.29.17
dcor==0.4
decorator==4.4.2
defusedxml==0.6.0
direpack==1.0.0
entrypoints==0.3
future==0.18.2
importlib-metadata==1.5.0
ipopt==0.1.9
ipykernel==5.1.4
ipython==7.13.0
ipython-genutils==0.2.0
ipywidgets==7.5.1
jedi==0.17.0
Jinja2==2.11.2
joblib==0.14.1
jsonschema==3.2.0
jupyter==1.0.0
jupyter-client==6.1.3
jupyter-console==6.1.0
jupyter-core==4.6.3
kiwisolver==1.2.0
llvmlite==0.32.1
MarkupSafe==1.1.1
matplotlib==3.2.1
mistune==0.8.4
mpmath==1.1.0
nbconvert==5.6.1
nbformat==5.0.6
notebook==6.0.3
numba==0.49.1
numpy==1.18.4
opencv-python==4.2.0.34
pandas==1.0.3
pandocfilters==1.4.2
parso==0.7.0
patsy==0.5.1
pexpect==4.8.0
pickleshare==0.7.5
Pillow==7.1.2
prometheus-client==0.7.1
prompt-toolkit==3.0.4
ptyprocess==0.6.0
Pygments==2.6.1
pyparsing==2.4.7
pyrsistent==0.16.0
python-dateutil==2.8.1
pytz==2020.1
pyzmq==18.1.1
qtconsole==4.7.3
QtPy==1.9.0
scikit-learn==0.22.2.post1
scipy==1.

In [ ]:
!wget -O a3_face_dataset.tar.gz https://www.dropbox.com/s/4nmsiafyvw0o5fx/a3_face_dataset.tar.gz?dl=0

In [ ]:
!tar xzf a3_face_dataset.tar.gz
!ls

In [ ]:
import glob
import matplotlib.pyplot as plt
from IPython.display import display, Image

# Display the first images from the dataset. You can also browse these by
# navigating through the notebooks file system!
sample_images = glob.glob('a3_face_dataset/*.jpg')[:5]
for file_path in sample_images:
  #display(Image(file_path))
    x = 1

In [ ]:
import cv2
import numpy as np

# Load images in greyscale.
image_paths = list(glob.glob('a3_face_dataset/*.jpg'))
images = np.stack([cv2.imread(str(x), cv2.IMREAD_GRAYSCALE) 
                   for x in image_paths])
print('Original Image Shape (N, H, W):', images.shape)

In [ ]:
import PIL
try:
    import Image
except ImportError:
    from PIL import Image
# Helper function for cropping the center of the image to remove the background
# and only keep the face.
def center_crop(img):
    h, w = img.shape
    center_y, center_x = h // 2, w // 2
    offset_y, offset_x = center_y // 2, center_x // 2
    return img[center_y - offset_y: center_y + offset_y,
               center_x - offset_x: center_x + offset_x]
  
# Center crop our images.
images = np.stack([center_crop(img) for img in images])
img_height, img_width = images.shape[1:3]
print('Cropped Image Shape (N, H, W):', images.shape)

# Visualize a few results.
for i in range(5):
  display(PIL.Image.fromarray(images[i]))

In [ ]:
# Flatten the images into a (N, HxW) sized array, where N is the number of 
# images and H, W are the dimensions of the image.
num_images = len(images)
images = images.reshape(num_images, -1)
print('Matrix Shape (N, H*W):', images.shape)

In [ ]:
import sklearn.decomposition as skd

In [ ]:
num_components = 4

pca = skd.PCA(n_components = num_components)
transformed = pca.fit_transform(images)

In [ ]:
import matplotlib.pyplot as plt

def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())

In [ ]:
projected = pca.inverse_transform(transformed)

plot_gallery(images[:30], range(30), img_height, img_width, 5, 6)
plot_gallery(projected[:30], range(30), img_height, img_width, 5, 6)

In [ ]:
from direpack import dicomo, ppdire

In [ ]:
pppca = ppdire(projection_index = dicomo, 
               pi_arguments = {'mode' : 'var'}, 
               n_components=num_components, 
               optimizer='grid', 
               optimizer_options={'ndir':100,'maxiter':100})

In [ ]:
%time pppca.fit(images[:5])

In [ ]:
import os

# Output directory.
os.makedirs('./eigenfaces/', exist_ok=True)

# Visualize the mean face.
print('Saving mean face...', end='')
mean_face = pca.mean_.reshape(img_height, img_width)
norm_img = cv2.normalize(mean_face, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
cv2.imwrite('./eigenfaces/mean_face.png', norm_img)
print('Done.')

# TODO: Visualize the first few eigenfaces, and save them as './eigenfaces/eigenface_{i}.png'.
# Don't forget to normalize the image like the the mean face example above.
print('Saving first ten eigen-faces to ./outputs/...', end='')
eigenfaces = pca.components_.reshape(num_components, img_height, img_width)[:10]
print(pca.components_.shape)
# --> Your code here <--
for i, img in enumerate(eigenfaces[:10]):
    norm_img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    cv2.imwrite('./eigenfaces/eigenface_{}.png'.format(i), norm_img)
print('Done.')

# Display the mean face and eigenfaces. You can also browse these by
# navigating through the notebooks file system!
print('Mean Face')
display(Image.open('./eigenfaces/mean_face.png'))

print('Eigenfaces')
for i in range(10):
  display(Image.open(f'./eigenfaces/eigenface_{i}.png'))